# Magnitude in Vega to AB conversion and extinction correction

Author: Ridha Fathima Mohideen Malik, Biagio de Simone

Updated: 19-04-2023

### Imports

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.io as pio

import shutil
import os
import re

from astropy.coordinates import SkyCoord
import astropy.units as u
import astropy.constants as const
from astropy.time import Time

import sys
sys.path.append('D:/grb/grbLC/')
import grblc
from grblc.convert.sfd import SFDQuery

import warnings
warnings.filterwarnings("ignore")

### String matching

In [3]:
# count function

def count(str1, str2): 
    '''
    Counts how much percentage two strings match. 
    ------
    INPUT:
    ------
    str1, str2: strings
    ------
    OUTPUT:
    ------
    match: percentage match
    '''
    
    #str1 = re.sub(r"[\u02BB\u02BC\u066C\u2018\u201A\u275B\u275C\u0027\u02B9\u02BB\u02BC\u02BE\u02C8\u02EE\u0301\u0313\u0315\u055A\u05F3\u07F4\u07F5\u1FBF\u2018\u2019\u2032\uA78C\uFF07]", "p", str1)
    #str2 = re.sub(r"[\u02BB\u02BC\u066C\u2018\u201A\u275B\u275C\u0027\u02B9\u02BB\u02BC\u02BE\u02C8\u02EE\u0301\u0313\u0315\u055A\u05F3\u07F4\u07F5\u1FBF\u2018\u2019\u2032\uA78C\uFF07]", "p", str2)
    
    diff = len(str1) - len(str2)
    if diff < 0:
      temp = str1
      str1 = str2
      str2 = temp
    else:
      pass

    c, j = 0,0
    for i in str1:    
        if str2.find(i)>= 0 and j == str1.find(i): 
            c += 1
        j+=1
    
    if len(str1)>0:
        match = c/len(str1)*100
    else:
        match = 0

    return match

def stripcount(str1, str2): 
    '''
    Counts how much percentage two strings match excluding special characters 
    ------
    INPUT:
    ------
    str1, str2: strings
    ------
    OUTPUT:
    ------
    match: percentage match excluding special characters
    '''

    str1 = ''.join(i for i in str1 if i.isalnum()).lower() ## removes special characters
    str2 = ''.join(i for i in str2 if i.isalnum()).lower()

    diff = len(str1) - len(str2)
    if diff < 0:
      str1 = str1 + ("-"*diff)
    elif diff > 0:
      str2 = str2 + ("-"*diff)
    else:
      pass
    
    c, j = 0,0
    for i in str1:    
        if str2.find(i)>= 0 and j == str1.find(i): 
            c += 1
        j+=1
    
    if len(str1)>0:
        match = c/len(str1)*100
    else:
        match = 0

    return match

In [4]:
def count_hst(str1, str2):
    wave1 = re.findall("\d+", str1)
    wave2 = re.findall("\d+", str2)
    if wave1 == wave2:
        str1="".join(re.findall("[a-zA-Z]+", str1))
        str2="".join(re.findall("[a-zA-Z]+", str2))
        match = count(str1, str2)
    else:
        match = 0

    return match

### Required files

In [5]:
path_convert = 'D:/grb/grbLC/grblc/convert/'

filters = pd.read_csv(path_convert+'filters.txt', sep='\s+', header=0, index_col=0, engine='python', encoding='ISO-8859-1')
adps = pd.read_csv(path_convert+'reddening.txt', sep='\t', header=0, index_col=0, engine='python', encoding='ISO-8859-1')
schafly = pd.read_csv(path_convert+'SF11_conversions.txt', sep='\t', header=0, index_col='lambda_eff')

### Filter matching

In [34]:
# function to match filter and telescope info to get band wavelength, zeropoint and extinction correction

def calibration(band: str, telescope: str):

    ## Step 1: initialising data input

    ## finding the bandpass and filter in data
    
    sep = ['-','_','.',',']

    j = 0
    for i in sep:
        if i in band:
            bandpass, filter = band.split(i)
            if filter.lower in ['unfiltered', 'clear']:
                filter = 'clear'
            else:
                if len(filter) > len(bandpass):
                    filter, bandpass = band.split(i)
            j += 1
    if j == 0:
        filter = band
        bandpass = ''

    ## assumptions

    assume_R = ['-', '—', '|', '\\', '/', '35', '145', 'P-', 'P—', 'P|', 'P\\', 'P/', 'P', 'polarised', 'polarized', 'unfiltered', 'clear', 'CR', 'lum', 'N', 'IR-cut', 'TR-rgb', 'RM'] # q

    for i in assume_R:
      if filter.casefold() == i:
        filter = 'Rc' ## Gendre's suggestion
    
    if filter == 'CV': ## CV clear calibrated as V
      filter ='V'
    
    if filter == 'BJ':
      filter = 'B'

    if filter == 'VJ':
      filter = 'V'        
        
    if filter == 'UJ':
      filter = 'U'
    
    if filter == 'BM':
      filter = 'B'    
    
    if filter == 'UM':
      filter = 'U'  

    ## formatting
    
    if filter == 'KS':
      filter = 'Ks'
    
    if filter == 'IC':
      filter = 'Ic'
    
    if filter == 'RC':
      filter = 'Rc'
    
    if filter == 'UB':
      filter = 'U'
    
    if "'" in filter:
        filter = filter.replace("'","p")
        
    if "*" in filter:
        filter = filter.replace("*","p")
    
    if "+" in telescope:
      telescope=telescope.split("+")[0]
    
    if "'" in filter:
        filter = filter.replace("'","p")
        
    if "*" in filter:
        filter = filter.replace("*","p")
    
    if "+" in telescope:
      telescope=telescope.split("+")[0]

    ## finding the observatory, telescope, instrument in data

    observatory, telescope, instrument = telescope.split('/') 

    if instrument == 'CCD':
      instrument = 'None'

    if "." in telescope:
      telescope=telescope.replace(".", ",")


    ## Step 2: checking if the data filter exists in grblc filters

    for id in filters.index:       
      grblc_fil = str(id).split(".")[-1]
      if len(filter) >= 5:
        filters.loc[id, 'match_fil'] = count_hst(grblc_fil, filter)
      else:
        if grblc_fil == filter:
          filters.loc[id, 'match_fil'] = 100
        else:
          filters.loc[id, 'match_fil'] = count(grblc_fil, filter)

    matched_fil = filters.loc[filters['match_fil'] == 100]
    if len(matched_fil) == 0 and len(filter) <= 2:
      matched_fil = filters.loc[filters['match_fil'] >= 50]
      if len(matched_fil) == 0:
        raise KeyError(
            f"No matching filters.")
    elif len(matched_fil) == 0 and len(filter) > 2:
       raise KeyError(
            f"No matching filters.")
      
    ## Step 3: finding exact matches in observatory, telescope, instrument

    probablefilters = []
    
    for id in matched_fil.index:
      
      grblc_obs, grblc_tel, grblc_ins, *__ = str(id).split(".")
        
      if grblc_obs.casefold() == observatory.casefold():

        matched_fil.loc[id, 'match_obs'] = 'found'

        match_tel =  count(grblc_tel.casefold(), telescope.casefold())
        match_ins =  count(grblc_ins.casefold(), instrument.casefold())

        if match_tel == 100:
          matched_fil.loc[id, 'match_tel'] = 1
          if match_ins == 100:
            matched_fil.loc[id, 'match_ins'] = match_ins
          elif match_ins >= 50:
            matched_fil.loc[id, 'match_ins'] = match_ins
          else:
            matched_fil.loc[id, 'match_ins'] = match_ins

        elif match_tel >= 50:
            matched_fil.loc[id, 'match_tel'] = 2
            matched_fil.loc[id, 'match_ins'] = match_ins

        else:
          matched_fil.loc[id, 'match_tel'] = 3
          matched_fil.loc[id, 'match_ins'] = None

      else:
         matched_fil.loc[id, 'match_obs'] = None
         matched_fil.loc[id, 'match_tel'] = None
         matched_fil.loc[id, 'match_ins'] = None

    matched_obs =  matched_fil.loc[matched_fil['match_obs'] == 'found']
    if len(matched_obs) != 0:
      matched_tel = matched_obs.loc[matched_obs['match_tel'] == np.max(matched_obs['match_tel'])]
      matched_tel =  matched_tel.sort_values(by=['match_ins'])
      probablefilters = list(matched_tel.index)

    ## Step 4: in case of no match, resort to generics

    #standard = ['Johnson', 'Cousins', 'Bessel', 'Special', 'Tyson', 'SDSS', 'SuperSDSS', 'Stromgren', 'MKO', 'UKIRT', 'UKIDSS', 'PS1']

    if len(probablefilters) == 0:
      for id in matched_fil.index:

        grblc_obs = str(id).split(".")[0]

        if grblc_obs.casefold()=='average':
          matched_fil.loc[id, 'match_status'] = 1

        elif grblc_obs.casefold()=='generic':
          matched_fil.loc[id, 'match_status'] = 2
          
        elif grblc_obs.casefold()=='gcpd':
          matched_fil.loc[id, 'match_status'] = 3

        elif grblc_obs.casefold()=='catalog':
          matched_fil.loc[id, 'match_status'] = 4

        else:
          matched_fil.loc[id, 'match_status'] = None

      matched_gen =  matched_fil.sort_values(by=['match_status'], na_position='last')
      probablefilters = list(matched_gen.index)

    correctfilter = probablefilters[0]
    
    try:
      lam = float(matched_fil.loc[correctfilter,'lambda_eff'])
    except TypeError:
      lam = float(matched_fil.loc[correctfilter,'lambda_eff'][0])
    
    lam_round = round(lam, -1)

    shift_toAB = matched_fil.loc[correctfilter,'mag_fromVega_toAB']

    try:
        coeff = schafly.loc[lam_round, '3.1']
        coeff_source = "Schafly+11"
    except KeyError:
        coeff = adps.loc[lam_round, 'coeff']
        coeff_source = "APDS+02"
    
    return lam, shift_toAB, coeff, correctfilter, coeff_source

In [7]:
calibration(band='J', telescope='KO/Keck-2(10m)/LRIS')

(12510.0, 'notfound', 0.795818, 'KO.Keck-1.NIRC.None.J', 'APDS+02')

In [8]:
calibration(band='F606W', telescope='Hubble/HST/STIS')

(5809.26, '0.0865', 2.80733, 'Hubble.HST.WFC1.None.F606W', 'APDS+02')

### Extinction correction

In [9]:
# getting color excess

def ebv(grb: str, ra="", dec=""):

    sfd = SFDQuery()

    if not (ra or dec):
        import astroquery.exceptions
        from astroquery.simbad import Simbad

        try:
            obj = Simbad.query_object(f"GRB {grb}")
            skycoord = SkyCoord(
                "{} {}".format(obj["RA"][0], obj["DEC"][0]), unit=(u.hourangle, u.deg)
            )
        except astroquery.exceptions.RemoteServiceError:
            raise astroquery.exceptions.RemoteServiceError(
                f"Couldn't find the position of GRB {grb}. Please supply RA and DEC manually."
            )
    else:
        skycoord = SkyCoord(f"{ra} {dec}", frame="icrs", unit=(u.hourangle, u.deg))

    # this grabs the degree of reddening E(B-V) at the given position in the sky.
    # see https://astronomy.swin.edu.au/cosmos/i/interstellar+reddening for an explanation of what this is
    ebv = sfd(skycoord)

    return ebv

### Conversion

In [10]:
# default shifts to AB

defaultshift_toAB=[["U",0.800527],
            ["B",-0.107512],
            ["BM",-0.107512],
            ["V",0.006521],
            ["VM",0.006521],
            ["R",0.190278],
            ["Unfiltered",0.190278],
            ["unfiltered",0.190278],
            ["Clear",0.190278],
            ["clear",0.190278],
            ["N",0.190278],
            ["Lum",0.190278],
            ["lum",0.190278],
            ["RM",0.190278],
            ["TR-rgb",0.190278],
            ["IR-cut",0.190278],
            ["CR",0.190278],
            ["CV",0.006521],
            ["I",0.431372],
            ["Rc",0.117],
            ["Ic",0.342],
            ["RC",0.117],
            ["IC",0.342],
            ["Bj",1.344],
            ["BJ",1.344],
            ["VJ",0.006521],
            ["UJ",0.800527],
            ["u",0.9],
            ["g",-0.125],
            ["g_Gunn",-0.013],
            ["r",0.119],
            ["r_Gunn",-0.226], #https://lweb.cfa.harvard.edu/~dfabricant/huchra/ay145/mags.html
            ["i",0.332],
            ["i_Gunn",-0.296], #https://lweb.cfa.harvard.edu/~dfabricant/huchra/ay145/mags.html
            ["z",0.494],
            ["z_Gunn",0.494], #https://lweb.cfa.harvard.edu/~dfabricant/huchra/ay145/mags.html
            ["up",0],
            ["u'",0],
            ["gp",-0.125],
            ["g'",-0.125],
            ["rp",0.119],
            ["r'",0.119],
            ["ip",0.332],
            ["i'",0.332],
            ["z'",0.494],
            ["zp",0.494],
            ["H",1.344],
            ["J",0.87],
            ["K",1.815],
            ["Ks",1.814],
            ["KS",1.814],
            ["Kp",1.84],
            ["K'",1.815], 
            ["Z",0.489],
            ["Y",0.591],
            ["q",0.190278],
            ["w",0.8],
            ["F220W",1.683496],
            ["F250W",1.495635],
            ["F330W",1.097],
            ["F344N",1.151053],
            ["F435W",-0.129],
            ["F475W",-0.122],
            ["F502N",-0.083596],
            ["F550M",0.024283],
            ["F555W",-0.03],
            ["F606W",0.063],
            ["F625W",0.14],
            ["F658N",0.376572],
            ["F660N",0.278863],
            ["F775W",0.364],
            ["F814W",0.4],
            ["F850LP",0.494],
            ["F892N",0.487505]]

In [11]:
# conversion for mag in AB and corrected for extinction

def toAB(
    grb: str,
    ra: str,
    dec: str,
    band: str,
    system: str,
    extcorr: str,
    telescope: str,
    mag: float
):

    try:
        lambda_x, shift_toAB, coeff, filter_id, coeff_source = calibration(band, telescope)
    except KeyError:
        raise KeyError(f"Band '{band}' of telescope '{telescope}' is not currently supported.")

    # get correction for galactic extinction to be added to magnitude if not already supplied
    if extcorr == 'y':
        mag_corr = mag    
    else:
        A_x = coeff * ebv(grb, ra, dec)
        mag_corr = mag - A_x

    if 'ab' or 'sdss' or 'panstarrs' in system.casefold:
        mag_corr = mag_corr
    else:
        if shift_toAB != 'notfound':
            mag_corr = mag_corr + float(shift_toAB)
                    
        else:
            for i in defaultshift_toAB:
                if band==i[0]:
                    shiftAB=i[1]
                    break
            mag_corr = mag_corr + float(shiftAB)

    return mag_corr, filter_id, coeff_source

In [12]:
# main conversion function to call
def correctGRB(
    grb: str,
    ra: str,
    dec: str,
    filename: str = None,
    save_in_folder: str = None,
    debug: bool = False,
):

    # assign column names and datatypes before importing
    dtype = {
        'time_sec': np.float64,
        'mag': np.float64,
        'mag_err': np.float64,
        'band': str,
        'system': str,
        'telescope': str,
        'extcorr': str,
        'source': str,
    }
    names = list(dtype.keys())

    # try to import magnitude table to convert
    try:
        global directory
        mag_table = pd.read_csv(
            filename,
            delimiter=r'\t+|\s+',
            names=names,
            dtype=dtype,
            index_col=None,
            header=0,
            engine='python',
            encoding='ISO-8859-1'
        )
    except ValueError as error:
        raise error
    except IndexError:
        raise ImportError(message=f"Couldn't find grb table at {filename}.")

    converted = {k: [] for k in ('time_sec', 'mag', 'mag_err', 'band', 'system', 'telescope', 'extcorr', 'source')}

    if debug:
        converted_debug = {
            k: []
            for k in (
                'time_sec',
                'mag_corr',
                'mag_init',
                'mag_err',
                'band',
                'band_match',
                'system_init',
                'system_final',
                'telescope',
                'extcorr',
                'coeff_source'
                'mag_source'
            )
        }

    for __, row in mag_table.iterrows():

        time_sec = row['time_sec']
        mag = row['mag']
        mag_err = row['mag_err']
        system = row['system']
        band = row['band']
        telescope = row['telescope']
        extcorr = row['extcorr']
        source = row['source']

        # attempt to convert a single magnitude to flux given a band, position in the sky, mag_err, and photon index
        try:
            mag_corr, filter_id, coeff_source = toAB(
                grb,
                ra,
                dec,
                band,
                system,
                extcorr,
                telescope,
                mag
                )
        except KeyError as error:
            print("KeyError")
            continue


        converted['time_sec'].append(time_sec)
        converted['mag'].append(mag_corr)
        converted['mag_err'].append(mag_err)
        converted['band'].append(band)
        converted['system'].append("AB")
        converted['telescope'].append(telescope)
        converted['extcorr'].append("y")
        converted['source'].append(source)

        # verbosity if you want it
        if debug:
            converted_debug['time_sec'].append(time_sec)
            converted_debug['mag_corr'].append(mag_corr)
            converted_debug['mag_init'].append(mag)
            converted_debug['mag_err'].append(mag_err)
            converted_debug['band_init'].append(band)
            converted_debug['band_match'].append(filter_id)
            converted_debug['system_init'].append(system)
            converted_debug['system_final'].append("AB")
            converted_debug['telescope'].append(system)
            converted_debug['extcorr'].append("y")
            converted_debug['coeff_source'].append(coeff_source)
            converted_debug['mag_source'].append(source)

    # after converting everything, go from dictionary -> DataFrame -> csv!
    if not debug:
        save_path = os.path.join(save_in_folder+'/', f"{grb}_magAB_extcorr.txt")
        pd.DataFrame.from_dict(converted).to_csv(save_path, sep='\t', index=False)
    else:
        save_path = os.path.join(save_in_folder+'/', f"{grb}_magAB_extcorr_DEBUG.txt")
        pd.DataFrame.from_dict(converted_debug).to_csv(save_path, sep='\t', index=False)


### Getting files

In [13]:
# Create a list of the path of all our grbs.

path = "D:/grb/sample/mag-30-05-2023/"

grblist = []
filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        filelist.append(os.path.join(root,file))
        names = file.split('_')
        grblist.append(names[0])

filedf=pd.DataFrame()
filedf.index = grblist
filedf['path'] = filelist

#filedf = filedf.sort_values(by='grb')

## sorting by date
#filedf1 = filedf[0:516]
#filedf2 = filedf[516::]
#filedf = pd.concat([filedf2, filedf1])
#filedf = filedf.reset_index(drop=True)

filedf.to_csv('magVega_to_AB.csv')
filedf.head()

,path
000131A,D:/grb/sample/mag-30-05-2023/000131A_mag.txt
000301C,D:/grb/sample/mag-30-05-2023/000301C_mag.txt
000418A,D:/grb/sample/mag-30-05-2023/000418A_mag.txt
000911A,D:/grb/sample/mag-30-05-2023/000911A_mag.txt
000926A,D:/grb/sample/mag-30-05-2023/000926A_mag.txt


In [14]:
radec = pd.read_csv('sample.csv', sep=',', header=0, index_col='grb')
radec.head()

,ra,dec,coorderr,z
grb,,,,
970228A,05h01m57s,+11° 46.4',3',0.6950
970508A,06h53m28s,+79° 17.4',3',0.8300
971214A,11h56m30s,+65° 12.0',4',3.4200
980329A,07h02m41s,+38° 50.7',3',3.5000
980425A,19h34m54s,-52° 49.9',8',0.0085


In [15]:
for radec_grb in radec.index:
    for files_grb in filedf.index:
        if files_grb == radec_grb:
            radec.loc[radec_grb, 'path'] = filedf.loc[files_grb, 'path']
            pass

radec.to_csv('radec.csv', sep=',')

In [16]:
radec.head()

,ra,dec,coorderr,z,path
grb,,,,,
970228A,05h01m57s,+11° 46.4',3',0.6950,D:/grb/sample/mag-30-05-2023/970228A_mag.txt
970508A,06h53m28s,+79° 17.4',3',0.8300,D:/grb/sample/mag-30-05-2023/970508A_mag.txt
971214A,11h56m30s,+65° 12.0',4',3.4200,D:/grb/sample/mag-30-05-2023/971214A_mag.txt
980329A,07h02m41s,+38° 50.7',3',3.5000,D:/grb/sample/mag-30-05-2023/980329A_mag.txt
980425A,19h34m54s,-52° 49.9',8',0.0085,D:/grb/sample/mag-30-05-2023/980425A_mag.txt


### Conversion Let's gooooo

**Parallel processing**

In [23]:
def bigloop(grb):

    Debug = False

    status_conv = False
    status_err = False

    path_results = 'mag-new-conv/'

    print(grb)
    
    if 1==1:#try:
        correctGRB(
            grb=grb,
            filename = radec.loc[grb,'path'],
            save_in_folder=path_results,
            ra=radec.loc[grb,'ra'],
            dec=radec.loc[grb,'dec'],
            debug=Debug
        )
        print(": Converted")
        status_conv = True

        if Debug:
            conv_file = path_results+grb+'_magAB_extcorr_DEBUG.txt'
        else:
            conv_file = path_results+grb+'_magAB_extcorr.txt'

        light = pd.read_csv(
            conv_file,
            delimiter=r"\t+|\s+",
            index_col=None,
            header=0,
            engine="python",
            encoding="ISO-8859-1"
        )
        
        fig = px.scatter(
            data_frame=light,
            x=np.log10(light['time_sec'].values),
            y=light['mag'].values,
            error_y= light['mag_err'].values,
            color=light['band'].values,
            hover_data=['source'],
            title=grb,
        )
        fig.update_layout(
            xaxis_title='log Time (s)',
            yaxis_title='AB magnitudes'
        )
        fig.update_yaxes(autorange="reversed")
        #pio.write_html(fig, file="mag-plots/"+grb+".html")
        
    #except:
        #count_err = True
        #shutil.copy(str(filedf.loc[grb,'path']),"mag-errorbox")
        
    return  grb, status_conv, status_err

In [20]:
todo = ['140606B','201103B', '051006A', '060108A']

In [21]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

num_cores = multiprocessing.cpu_count()
inputs = tqdm(todo)

if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(bigloop)(i) for i in inputs)


100%|██████████| 4/4 [00:00<00:00, 3970.94it/s]


In [22]:
processed_list

[('140606B', False, False),
 ('201103B', False, False),
 ('051006A', False, False),
 ('060108A', False, False)]

In [30]:
bigloop('051006A')

051006A
: Converted


FileNotFoundError: [Errno 2] No such file or directory: 'mag-plots\\051006A.html'

In [31]:
bigloop('060108A')

060108A
: Converted


FileNotFoundError: [Errno 2] No such file or directory: 'mag-plots\\060108A.html'

In [36]:
bigloop('201103B')

201103B
['MAO', 'AZT-22(1.5m)', 'SITe2k']
['MAO', 'AZT-22(1.5m)', 'SITe2k']
['Xinjiang', 'NEXT(0.6m)', 'CCD']
['ISON-K', 'RC-360', 'CCD']
['PAO', 'VLT-UT3-Melipal', 'X-SHOOTER']
['NGSO', 'Swift', 'UVOT']
['Xinglong', '2.16m', 'BFOSC']
['Sayan', 'AZT-33IK(1.6m)', 'E2V-CCD30-11']
['Yunnan', 'GMG(2.4m)', 'YFOSC']
['TSAO', 'Zeiss-1000', 'CCD']
['ATO', 'AZT-20', 'KL400']
['IAO', 'GROWTH-India(0.7m)', 'AndoriKon-XL']
['Yunnan', 'GMG(2.4m)', 'YFOSC']
['Sayan', 'AZT-33IK(1.6m)', 'E2V-CCD30-11']
['DOAO', '1m', 'CCD']
['Sayan', 'AZT-33IK(1.6m)', 'E2V-CCD30-11']
['IAO', 'GROWTH-India(0.7m)', 'AndoriKon-XL']
['KCT', 'AZT-8(0.36m)', 'CMOS']
['SSO', 'KMTNet', 'CCD']
['Sayan', 'AZT-33IK(1.6m)', 'E2V-CCD30-11']
['Abastumani', 'AS-32(0.7m)', 'PL4240-CCD']
['CrAO', 'ZTSH(2.6m)', 'FLI-PL4240']
['SAO-RAS', 'Zeiss-1000', 'EEV-4240-CCD']
['PO', 'Oschin(1.2m)', 'ZTF']
['PO', 'Oschin(1.2m)', 'ZTF']
['PO', 'Oschin(1.2m)', 'ZTF']
['PO', 'Oschin(1.2m)', 'ZTF']
['PO', 'Oschin(1.2m)', 'ZTF']
['PO', 'Oschin(1.2m)',

FileNotFoundError: [Errno 2] No such file or directory: 'mag-plots\\201103B.html'

**Checking errors**

In [16]:
# Create a list of the path of all our grbs.

path = "mag-sys-extcorr-19-04-2023"


grblist = []
filelist = []

for root, dirs, files in os.walk(path):
    for file in files:
        filelist.append(os.path.join(root,file))
        names = file.split('_')
        grblist.append(names[0])

convdf=pd.DataFrame()
convdf.index = grblist
convdf['path'] = filelist

In [26]:
for radec_grb in radec.index:
    for conv_grb in convdf.index:
        if conv_grb == radec_grb:
            radec.loc[radec_grb, 'conv'] = True
            radec.loc[radec_grb, 'path_conv'] = convdf.loc[conv_grb, 'path']
            pass

radec.to_csv('radec_output.csv', sep=',')

In [27]:
errordf = radec[radec['conv'] != True]

In [29]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

num_cores = multiprocessing.cpu_count()
inputs = tqdm(errordf.index)

if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(bigloop)(i) for i in inputs)

100%|██████████| 43/43 [00:09<00:00,  4.64it/s]


In [39]:
Debug = False

count_conv = 0
count_err = 0

path_results = 'mag-sys-extcorr-19-04-2023/'
error = ['210619B']

for grb in error: # ended on 17

    print(grb)
    
    if 1==1:#try:
        correctGRB(
            grb=grb,
            filename = radec.loc[grb,'path'],
            save_in_folder=path_results,
            ra=radec.loc[grb,'ra'],
            dec=radec.loc[grb,'dec'],
            debug=Debug
        )
        print(": Converted")
        count_conv += 1

        if Debug:
            conv_file = path_results+grb+'_magAB_extcorr_DEBUG.txt'
        else:
            conv_file = path_results+grb+'_magAB_extcorr.txt'

        light = pd.read_csv(
            conv_file,
            delimiter=r"\t+|\s+",
            index_col=None,
            header=0,
            engine="python",
            encoding="ISO-8859-1"
        )
        
        fig = px.scatter(
            data_frame=light,
            x=np.log10(light['time_sec'].values),
            y=light['mag'].values,
            error_y= light['mag_err'].values,
            color=light['band'].values,
            hover_data=['source'],
            title=grb,
        )
        fig.update_layout(
            xaxis_title='log Time (s)',
            yaxis_title='AB magnitudes'
        )
        fig.update_yaxes(autorange="reversed")
        pio.write_html(fig, file="mag-plots/"+grb+".html")  

print("Converted: ",count_conv)
print("Error: ",count_err)

210619B
: Converted
Converted:  1
Error:  0
